### Possible next steps...

- **Descriptive Analysis**: Start with a broad overview. Calculate basic statistics (mean, median, standard deviation, etc.) for each category and stat type. This will give you a sense of the data's central tendencies and variability. Visualize these stats through histograms, box plots, and density plots to understand the distribution of each statistic.

- **Correlation Analysis**: Investigate how different stats correlate with winning percentage and each other. This could help identify key factors contributing to a team's success. Heatmaps and scatter plots are great for visualizing these relationships.

- **Trend Analysis**: Analyze trends over the years. This could involve looking at how the importance of certain stats (like three-point shooting or turnovers) has evolved. Time series plots can be helpful here, showing how averages for significant stats have changed over time.

- **Comparative Analysis**: Compare stats between teams that have historically performed well in the tournament versus those that haven’t. This might reveal certain statistical benchmarks or thresholds that successful teams typically meet.

- **Cluster Analysis**: Use clustering techniques to group teams with similar statistical profiles. This could reveal archetypes of teams (e.g., defensive powerhouses, high-scoring teams) and how well each archetype tends to perform in the tournament.

- **Predictive Modeling**: More complex approach, building predictive models (like logistic regression for binary outcomes, or more sophisticated machine learning models) using historical data to predict tournament outcomes can be very insightful. Use your stats as features to predict outcomes such as game winners, tournament progression, or even identify potential upsets.

- **Incorporating KenPom Stats**: KenPom stats are highly regarded in college basketball analytics. Compare your findings with KenPom’s efficiency ratings, tempo, and other advanced metrics. This could validate your analysis or reveal new insights.

- **Visualization and Dashboarding**: Develop interactive dashboards using tools like Tableau, Power BI, or even Python libraries (Dash by Plotly, for instance) to make analysis accessible. This can help in exploring data dynamically, comparing teams, and identifying patterns more intuitively.

- **Narrative and Storytelling**: Use analysis to tell stories. This could involve identifying underdog teams with the potential to make deep runs, powerhouse teams that might be vulnerable, or key matchups to watch based on statistical matchups.

In [8]:
import pandas as pd
import numpy as np

In [9]:
kp14 = pd.read_csv("../data/Kenpom/2013-14_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp15 = pd.read_csv("../data/Kenpom/2014-15_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp16 = pd.read_csv("../data/Kenpom/2015-16_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp17 = pd.read_csv("../data/Kenpom/2016-17_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp18 = pd.read_csv("../data/Kenpom/2017-18_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp19 = pd.read_csv("../data/Kenpom/2018-19_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp20 = pd.read_csv("../data/Kenpom/2019-20_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp21 = pd.read_csv("../data/Kenpom/2020-21_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp22 = pd.read_csv("../data/Kenpom/2021-22_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp23 = pd.read_csv("../data/Kenpom/2022-23_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)
kp24 = pd.read_csv("../data/Kenpom/2023-24_kenpom-stats.csv").sort_values(by="adj_eff_margin", ascending=False).head(10)

In [10]:
kp14

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
149,Louisville,AAC,2014,31,6,30.41,118.9,88.5,67.2,-0.045,4.31,107.0,102.7,-4.65,4.0
11,Arizona,Pac-12,2014,33,5,30.11,116.5,86.4,62.8,0.014,9.30,109.9,100.6,1.62,1.0
87,Florida,SEC,2014,36,3,28.57,116.5,88.0,61.3,0.053,9.02,109.4,100.4,2.39,1.0
327,Virginia,ACC,2014,30,7,26.46,114.8,88.3,59.5,0.012,8.67,109.4,100.8,1.57,1.0
343,Wisconsin,Big Ten,2014,30,8,25.89,122.0,96.1,62.4,0.019,11.44,110.2,98.8,3.33,2.0
339,Wichita St,MVC,2014,35,1,25.36,117.8,92.4,63.5,-0.009,1.04,105.8,104.8,2.89,1.0
132,Kansas,Big 12,2014,25,10,24.60,119.3,94.7,66.6,-0.051,12.13,112.2,100.0,9.47,2.0
73,Duke,ACC,2014,26,9,24.25,124.7,100.4,64.6,-0.035,8.76,109.2,100.5,2.40,3.0
169,Michigan St,Big Ten,2014,29,9,23.92,118.6,94.7,64.4,-0.003,10.44,110.2,99.7,0.57,4.0
286,Tennessee,SEC,2014,24,13,23.69,117.9,94.2,61.6,-0.099,8.12,108.9,100.8,1.46,11.0


In [47]:
kp14.describe()

,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
count,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000
mean,2014.0,29.900000,7.100000,26.326000,118.700000,92.370000,63.390000,-0.014400,8.323000,109.220000,100.910000,2.105000,3.00000
std,0.0,4.067486,3.510302,2.521037,2.860458,4.458213,2.388607,0.043689,3.327158,1.765597,1.687503,3.422995,3.05505
min,2014.0,24.000000,1.000000,23.690000,114.800000,86.400000,59.500000,-0.099000,1.040000,105.800000,98.800000,-4.650000,1.00000
25%,2014.0,26.750000,5.250000,24.337500,116.825000,88.350000,61.800000,-0.042500,8.257500,108.975000,100.100000,1.487500,1.00000
50%,2014.0,30.000000,7.500000,25.625000,118.250000,93.300000,63.150000,-0.006000,8.890000,109.400000,100.550000,2.005000,2.00000
75%,2014.0,32.500000,9.000000,28.042500,119.200000,94.700000,64.550000,0.013500,10.155000,110.125000,100.800000,2.767500,3.75000
max,2014.0,36.000000,13.000000,30.410000,124.700000,100.400000,67.200000,0.053000,12.130000,112.200000,104.800000,9.470000,11.00000


In [11]:
kp15

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
137,Kentucky,SEC,2015,38,1,36.91,121.3,84.4,62.4,0.048,8.58,108.7,100.1,3.13,1.0
343,Wisconsin,Big Ten,2015,36,4,33.72,129.0,95.2,58.7,0.015,11.03,109.8,98.8,5.30,1.0
73,Duke,ACC,2015,35,4,32.48,124.5,92.0,65.8,0.067,11.17,110.7,99.5,1.31,1.0
11,Arizona,Pac-12,2015,34,4,32.36,119.9,87.5,65.8,-0.033,7.37,108.3,100.9,0.09,2.0
326,Villanova,Big East,2015,33,3,30.65,122.7,92.0,64.2,0.031,7.67,108.2,100.6,0.23,1.0
327,Virginia,ACC,2015,30,4,30.06,115.6,85.5,58.0,0.023,8.21,108.5,100.3,0.46,2.0
102,Gonzaga,WCC,2015,35,3,27.78,121.7,93.9,64.7,0.018,4.37,107.3,102.9,3.00,2.0
317,Utah,Pac-12,2015,26,9,27.24,116.4,89.2,60.5,-0.054,8.19,108.8,100.6,0.60,5.0
214,Notre Dame,ACC,2015,32,6,24.96,125.4,100.4,63.4,0.062,7.59,107.7,100.1,-11.62,3.0
185,N Carolina,ACC,2015,26,12,23.85,119.1,95.3,69.2,-0.026,12.76,111.1,98.3,7.64,4.0


In [12]:
kp16

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
326,Villanova,Big East,2016,35,5,32.01,122.8,90.8,66.0,0.023,10.74,110.5,99.7,2.98,2.0
185,N Carolina,ACC,2016,33,7,29.82,124.4,94.6,70.5,-0.055,11.45,111.7,100.2,3.13,1.0
132,Kansas,Big 12,2016,33,5,29.67,120.0,90.4,69.0,0.044,12.61,111.4,98.8,3.62,1.0
327,Virginia,ACC,2016,29,8,29.64,120.6,91.0,61.3,-0.040,12.67,112.1,99.5,4.98,1.0
169,Michigan St,Big Ten,2016,29,6,27.97,123.4,95.5,66.5,-0.012,7.19,107.5,100.3,0.40,2.0
137,Kentucky,SEC,2016,27,9,25.14,121.8,96.6,67.4,-0.046,8.37,108.6,100.3,3.19,4.0
149,Louisville,ACC,2016,23,8,24.68,113.4,88.8,66.1,-0.025,7.49,110.4,102.9,-7.40,NaN
333,W Virginia,Big 12,2016,26,9,24.45,115.3,90.9,69.5,0.003,11.24,110.9,99.6,0.72,3.0
235,Purdue,Big Ten,2016,26,9,23.85,116.9,93.1,67.9,-0.046,7.43,109.1,101.7,-2.67,5.0
219,Oklahoma,Big 12,2016,29,8,23.78,117.9,94.1,69.8,0.082,13.05,111.5,98.5,2.70,2.0


In [13]:
kp17

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
102,Gonzaga,WCC,2017,37,2,32.05,118.4,86.3,70.1,0.020,2.99,106.2,103.3,1.01,1.0
326,Villanova,Big East,2017,32,4,29.88,122.4,92.5,64.1,0.010,9.33,109.8,100.5,3.55,1.0
185,N Carolina,ACC,2017,33,7,28.22,120.7,92.5,71.3,0.037,12.49,112.0,99.5,3.87,1.0
137,Kentucky,SEC,2017,32,6,27.72,119.1,91.4,72.0,0.006,10.98,110.5,99.5,3.74,2.0
87,Florida,SEC,2017,27,9,27.50,116.9,89.5,69.0,-0.041,12.46,110.3,97.8,8.19,4.0
132,Kansas,Big 12,2017,31,5,27.45,121.9,94.5,70.2,0.067,12.43,111.2,98.8,2.39,1.0
333,W Virginia,Big 12,2017,28,9,27.17,116.5,89.4,69.9,-0.043,10.84,110.1,99.2,-7.97,4.0
339,Wichita St,MVC,2017,31,5,26.14,118.7,92.6,68.0,-0.021,1.50,105.0,103.5,-0.32,10.0
149,Louisville,ACC,2017,25,9,25.51,117.2,91.7,67.9,-0.023,11.14,111.6,100.5,4.20,2.0
223,Oregon,Pac-12,2017,33,6,25.02,118.2,93.2,67.0,0.034,8.05,109.2,101.2,0.51,3.0


In [14]:
kp18

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
326,Villanova,Big East,2018,36,4,33.76,127.8,94.0,68.7,-0.018,11.31,111.3,100.0,3.98,1.0
327,Virginia,ACC,2018,31,3,29.53,115.2,85.6,59.4,0.048,9.24,110.0,100.7,-0.08,1.0
73,Duke,ACC,2018,29,8,28.86,122.4,93.6,69.7,-0.034,10.60,110.4,99.8,4.09,2.0
47,Cincinnati,AAC,2018,31,5,26.60,113.4,86.8,64.5,-0.019,3.46,106.9,103.4,-4.21,2.0
235,Purdue,Big Ten,2018,30,7,26.27,122.7,96.4,67.2,0.006,9.02,109.1,100.1,0.69,2.0
169,Michigan St,Big Ten,2018,30,5,25.41,119.2,93.8,67.0,0.038,6.64,108.6,101.9,-5.03,3.0
168,Michigan,Big Ten,2018,33,8,24.20,114.7,90.5,64.9,0.052,10.32,111.4,101.1,-4.83,3.0
185,N Carolina,ACC,2018,26,11,23.59,120.3,96.7,71.2,-0.011,13.26,111.6,98.3,7.58,2.0
132,Kansas,Big 12,2018,31,8,23.49,120.9,97.4,68.8,0.076,12.41,111.7,99.3,3.55,1.0
102,Gonzaga,WCC,2018,32,5,23.03,118.1,95.1,69.4,0.009,1.57,106.4,104.8,3.12,4.0


In [15]:
kp19

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
329,Virginia,ACC,2019,35,3,34.22,123.4,89.2,59.4,0.050,11.18,109.2,98.1,-3.24,1.0
103,Gonzaga,WCC,2019,33,4,32.85,124.5,91.6,70.2,-0.001,4.46,106.9,102.5,1.87,1.0
170,Michigan St,Big Ten,2019,32,7,30.81,121.0,90.2,66.9,0.001,13.67,110.6,96.9,3.24,2.0
74,Duke,ACC,2019,32,6,30.62,120.0,89.3,72.1,0.018,12.85,110.7,97.8,5.08,1.0
292,Texas Tech,Big 12,2019,31,7,30.03,114.1,84.1,66.6,0.004,11.18,109.8,98.7,-5.39,3.0
169,Michigan,Big Ten,2019,30,7,28.32,114.5,86.2,64.8,-0.001,11.27,109.2,98.0,-5.42,2.0
187,N Carolina,ACC,2019,29,7,27.69,119.7,92.0,74.3,0.024,12.60,110.7,98.1,5.69,1.0
138,Kentucky,SEC,2019,30,7,27.57,117.6,90.1,65.6,0.033,11.32,110.7,99.4,1.75,2.0
237,Purdue,Big Ten,2019,26,10,26.81,122.5,95.6,65.9,-0.015,13.16,110.3,97.1,4.09,3.0
288,Tennessee,SEC,2019,31,6,26.24,122.7,96.5,67.8,0.034,10.69,109.7,99.0,0.92,2.0


In [16]:
kp20

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
133,Kansas,Big 12,2020,28,3,30.23,115.8,85.5,67.3,0.040,12.66,107.4,94.7,9.58,1.0
103,Gonzaga,WCC,2020,31,2,26.95,121.3,94.4,71.9,0.050,2.42,103.5,101.0,-2.09,1.0
20,Baylor,Big 12,2020,26,4,25.49,113.5,88.1,66.2,0.016,10.20,106.4,96.2,1.38,1.0
66,Dayton,A-10,2020,29,2,24.93,119.1,94.1,67.6,0.002,2.74,104.1,101.3,-0.74,1.0
74,Duke,ACC,2020,25,6,24.62,115.7,91.1,72.0,-0.009,7.28,106.0,98.7,2.60,3.0
262,San Diego St,MWC,2020,30,2,24.48,115.1,90.6,64.6,-0.008,2.98,105.3,102.3,-1.80,2.0
171,Michigan St,Big Ten,2020,22,9,24.03,115.2,91.2,69.1,-0.012,12.04,108.6,96.5,1.83,3.0
221,Ohio St,Big Ten,2020,21,10,22.23,114.3,92.0,66.2,-0.038,11.10,108.0,96.9,-1.31,5.0
150,Louisville,ACC,2020,24,7,21.36,114.5,93.1,67.1,-0.003,7.10,105.3,98.2,-1.09,4.0
335,W Virginia,Big 12,2020,21,10,21.26,108.3,87.1,69.4,-0.028,11.28,107.2,95.9,5.80,6.0


In [17]:
kp21

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
104,Gonzaga,WCC,2021,31,1,36.48,126.4,89.9,73.8,0.022,11.35,108.1,96.8,6.21,1.0
20,Baylor,Big 12,2021,28,2,33.87,125.0,91.1,67.4,0.046,13.25,108.7,95.5,-2.83,1.0
171,Michigan,Big Ten,2021,23,5,29.67,117.6,87.9,66.6,0.007,16.44,111.7,95.3,2.51,1.0
120,Illinois,Big Ten,2021,24,7,29.06,118.0,88.9,70.5,0.022,18.17,112.3,94.1,10.44,1.0
114,Houston,AAC,2021,28,4,28.75,118.3,89.6,64.4,0.018,9.21,105.7,96.5,2.11,2.0
321,USC,Pac-12,2021,25,8,27.58,115.7,88.1,67.0,0.001,16.27,111.1,94.8,6.28,6.0
126,Iowa,Big Ten,2021,22,9,26.87,123.5,96.7,69.9,-0.027,16.41,110.4,94.0,-4.90,2.0
56,Colorado,Pac-12,2021,23,9,25.10,116.3,91.2,66.1,-0.025,15.43,110.3,94.9,3.82,5.0
4,Alabama,SEC,2021,26,7,25.09,112.9,87.8,73.3,0.020,14.80,110.1,95.3,9.45,2.0
153,Loyola-Chi,MVC,2021,26,5,24.32,111.3,87.0,63.7,-0.024,6.27,105.6,99.4,4.81,8.0


In [18]:
kp22

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
104,Gonzaga,WCC,2022,28,4,30.95,120.7,89.7,72.5,-0.039,4.24,104.6,100.3,-2.88,1.0
114,Houston,AAC,2022,32,6,27.70,116.8,89.1,63.6,-0.012,5.89,105.5,99.6,0.18,5.0
134,Kansas,Big 12,2022,34,6,27.49,119.2,91.7,69.3,0.041,12.60,108.0,95.4,4.13,1.0
20,Baylor,Big 12,2022,27,7,27.04,118.0,91.0,67.4,0.004,10.96,107.4,96.5,-1.89,1.0
11,Arizona,Pac-12,2022,33,4,25.96,118.6,92.6,72.2,0.048,6.45,105.9,99.5,-0.78,1.0
139,Kentucky,SEC,2022,26,8,25.72,120.0,94.3,67.5,-0.043,8.88,106.5,97.6,-2.72,2.0
295,Texas Tech,Big 12,2022,27,10,25.64,110.7,85.0,66.5,-0.031,10.98,107.8,96.8,-8.24,3.0
75,Duke,ACC,2022,32,7,25.15,121.1,95.9,67.3,-0.010,8.22,107.8,99.6,-4.58,2.0
291,Tennessee,SEC,2022,27,8,25.10,111.5,86.4,67.4,0.029,11.39,108.5,97.1,4.85,3.0
333,Villanova,Big East,2022,30,8,24.56,117.5,92.9,62.3,0.060,11.46,109.1,97.6,8.65,2.0


In [19]:
kp23

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin,tourney_seed
59,Connecticut,Big East,2023,31,8,29.86,120.8,90.9,66.6,-0.056,10.39,109.7,99.3,-1.75,4.0
114,Houston,AAC,2023,33,4,27.72,118.1,90.4,63.5,0.023,4.56,107.2,102.6,-0.03,1.0
322,UCLA,Pac-12,2023,31,6,27.29,115.2,88.0,66.1,-0.009,8.70,109.2,100.5,0.15,2.0
4,Alabama,SEC,2023,31,6,27.28,115.5,88.2,72.6,0.057,11.07,110.2,99.2,10.46,1.0
296,Texas,Big 12,2023,29,9,24.43,116.5,92.1,68.8,0.017,11.91,111.1,99.2,-0.76,2.0
295,Tennessee,SEC,2023,25,11,23.80,111.3,87.5,65.1,-0.063,8.69,109.2,100.5,2.76,4.0
240,Purdue,Big Ten,2023,29,6,23.24,117.7,94.5,64.2,0.021,9.17,110.1,100.9,0.71,1.0
104,Gonzaga,WCC,2023,31,6,23.02,122.3,99.3,70.3,0.080,8.26,110.1,101.9,7.81,3.0
134,Kansas,Big 12,2023,28,8,22.85,114.6,91.8,69.2,0.082,13.38,111.0,97.6,6.59,1.0
160,Marquette,Big East,2023,29,7,22.38,119.1,96.7,68.1,0.009,9.31,109.5,100.2,-1.15,2.0


In [20]:
kp24

,team,conf,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin
113,Houston,Big 12,2024,26,3,31.83,120.5,88.7,64.1,0.034,9.31,111.0,101.7,-1.36
240,Purdue,Big Ten,2024,26,3,29.65,126.4,96.7,68.1,0.051,12.32,113.4,101.1,10.41
59,Connecticut,Big East,2024,25,3,29.60,125.2,95.6,64.8,0.043,7.39,110.3,102.9,-4.46
11,Arizona,Pac-12,2024,23,6,28.03,123.2,95.2,72.8,-0.030,9.97,112.0,102.1,9.96
294,Tennessee,SEC,2024,23,6,27.83,118.8,90.9,70.1,-0.029,11.17,113.7,102.6,7.74
17,Auburn,SEC,2024,22,7,27.43,120.1,92.7,69.8,-0.081,8.66,111.6,102.9,1.32
75,Duke,ACC,2024,23,6,25.71,122.1,96.4,67.4,-0.056,7.01,109.9,102.9,-0.52
4,Alabama,SEC,2024,20,9,25.01,127.3,102.3,73.0,-0.027,12.95,114.2,101.3,8.54
126,Iowa St,Big 12,2024,23,6,23.98,113.6,89.6,68.2,0.016,7.69,109.8,102.2,-7.93
190,N Carolina,ACC,2024,23,6,23.97,117.3,93.4,70.9,-0.030,10.39,112.2,101.8,6.67


In [48]:
kp24.describe()

,year,wins,losses,adj_eff_margin,adj_off_eff,adj_def_eff,adj_tempo,luck,sos_adj_eff_margin,opp_adj_off_eff,opp_adj_def_eff,nc_sos_adj_eff_margin
count,10.0,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,2024.0,23.400000,5.50000,27.304000,121.450000,94.150000,68.920000,-0.010900,9.686000,111.810000,102.150000,3.037000
std,0.0,1.837873,1.95789,2.625673,4.279992,4.011442,3.009171,0.044398,2.054287,1.587066,0.670406,6.491901
min,2024.0,20.000000,3.00000,23.970000,113.600000,88.700000,64.100000,-0.081000,7.010000,109.800000,101.100000,-7.930000
25%,2024.0,23.000000,3.75000,25.185000,119.125000,91.350000,67.575000,-0.030000,7.932500,110.475000,101.725000,-1.150000
50%,2024.0,23.000000,6.00000,27.630000,121.300000,94.300000,69.000000,-0.028000,9.640000,111.800000,102.150000,3.995000
75%,2024.0,24.500000,6.00000,29.207500,124.700000,96.200000,70.700000,0.029500,10.975000,113.100000,102.825000,8.340000
max,2024.0,26.000000,9.00000,31.830000,127.300000,102.300000,73.000000,0.051000,12.950000,114.200000,102.900000,10.410000
